<a href="https://colab.research.google.com/github/Tekleab15/Regularized_Auto_Encoder/blob/main/RAE_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularized Autoencoder Implementation
This notebook demonstrates the implementation of a Regularized Autoencoder (RAE) using backpropagation, inspired by the methodologies described in "The Neural Coding Framework for Learning Generative Models".



**Dataset Loading and Preprocessing: **all the four datasets used in the paper (MNIST, KMNIST, FMNIST, CalTech101)

# 1. Loading MNIST dataset


In [ ]:
# loading mnist dataset from the open online keras datasets
from tensorflow.keras.datasets import mnist

# load the dataset as it's splitted into a training and testing sets
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()
print("Dataset loaded successfully!")